In [1]:
#import statements
import json
import keras
import numpy as np
import random
import re
import spacy

from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
from tqdm import tqdm

In [2]:
# get training dataset 
with open("train-data-prepared.json", "r") as f:
    train_data = json.load(f)
# get training dataset 
with open("val-data-prepared.json", "r") as f:
    val_data = json.load(f)
# get testing dataset 
with open("val-data-prepared.json", "r") as f:
    test_data = json.load(f)


In [3]:
#create spacy object
nlp_english = spacy.load("en_core_web_sm")
#create Stemmer object
stemmer = PorterStemmer()

In [4]:
entire_dataset = {
    'train_ids': [thread["id"] for thread in train_data],
    'train_posts': [thread["preceding_posts"] for thread in train_data],
    'train_label': [thread["label"] for thread in train_data],
    
    'val_ids': [thread["id"] for thread in val_data],
    'val_posts': [thread["preceding_posts"] for thread in val_data],
    'val_label': [thread["label"] for thread in val_data],
    
    'test_ids': [thread["id"] for thread in test_data],
    'test_posts': [thread["preceding_posts"] for thread in test_data],
    'test_label': [thread["label"] for thread in test_data],
}

print(entire_dataset.keys())

dict_keys(['train_ids', 'train_posts', 'train_label', 'val_ids', 'val_posts', 'val_label', 'test_ids', 'test_posts', 'test_label'])


In [5]:
#for thread in val_data:
#    authors = []
#    for comment in thread["preceding_posts"]:
#        if comment["author_name"] not in authors:
#            authors.append(comment["author_name"])
#    
#    if len(authors) >= 2:
#        print(thread["id"])

In [6]:
# remove punctuation, space, urls from text
def clean_text(text):
    text = re.sub('[^A-Za-z0-9]+', ' ', text)
    parsed_text = nlp_english(text)
    clean_text = []
    for token in parsed_text:
        stop_flag = (token.is_punct or token.is_space or  
                 token.like_url)
        if (not stop_flag):
            clean_text.append(token.text.lower())
    return clean_text

def stem_text(text):
    return [stemmer.stem(word) for word in clean_text(text)]

print(stem_text("idiot stupid ass crazy screw you !"))
print(clean_text("> a) right, because women are non-sexual creatures who would never use prostitutes themselves\n\ni think you vastly overestimate the number of women that pay for sex..."))

['idiot', 'stupid', 'ass', 'crazi', 'screw', 'you']
['a', 'right', 'because', 'women', 'are', 'non', 'sexual', 'creatures', 'who', 'would', 'never', 'use', 'prostitutes', 'themselves', 'i', 'think', 'you', 'vastly', 'overestimate', 'the', 'number', 'of', 'women', 'that', 'pay', 'for', 'sex']


In [7]:
# checks if the OP has addressed the other person in some way
def check_insults(text):
    return [
        1 if "ass" in text else 0,
        1 if "idiot" in text else 0,
        1 if "screw" in text else 0,
        1 if "stupid" in text else 0
    ]
        
#print(check_insults(clean_text("> a) right, because women are non-sexual creatures who would never use prostitutes themselves\n\ni think you vastly overestimate the number of women that pay for sex...")))

In [8]:
# should return one feature vector for one string
# features -> Author's turn, length of argument, insults, sentiment
def gather_data(thread, train_data_flag = False):
    returnObj = {}
    for i in range(len(thread["preceding_posts"])):
        comment_data = {}
        comment = thread["preceding_posts"][i]
        # clean text
        comment_data["text"] = stem_text(comment["body"])
        # author's identity: 
        # Assuming 3 dialogues, the authors will have indices 0,1,2
        # 0,2 would be the OP
        # 1 would be the reply
        comment_data["author_turn_vec"] = [i % 2]
        # Length just in case of Godwin's Law
        comment_data["char_length_vec"] = [len("".join(clean_text(comment["body"])))]
        # check for some common insults
        comment_data["insults_vec"] = check_insults(comment_data["text"])
        # get sentiment
        sentiment = TextBlob(' '.join(comment_data["text"])).sentiment
        comment_data["sentiment"] =  [sentiment.polarity, sentiment.subjectivity]
        feature_vec = comment_data["author_turn_vec"] + comment_data["char_length_vec"] + comment_data["insults_vec"] + comment_data["sentiment"]
        
        returnObj[" ".join(comment_data["text"])] = feature_vec
        
    return returnObj

In [9]:
# print features for some 10 tuples
for thread in train_data[:5]:
    print(gather_data(thread))
    print()

{'a right becaus women are non sexual creatur who would never use prostitut themselv i think you vastli overestim the number of women that pay for sex': [0, 132, 0, 0, 0, 0, 0.39285714285714285, 0.6845238095238095], 'becaus it s illeg in our realiti vs the propos realiti that me op and everyon els on thi thread are comment about keep up qwert': [1, 113, 0, 0, 0, 0, 0.0, 0.0], 'i live in a nation were it is complet legal to pay for sex surround by other nation where it is also mostli legal guess what no woman pay for sex and you ll notic that it be illeg in the us doesn t stop men from go to prostitut keep up bunchanumb edit for context it s a first world nation with consider more liber attitud regard sex then the us': [0, 305, 0, 0, 0, 0, 0.19356060606060607, 0.3513888888888889]}

{'becaus make prostitut legal make it veri much more difficult to polic thi as ha been found repeatedli in countri around the world the women who are brought into the countri can t be question becaus there is

In [10]:
entire_dataset["train_prep"] = [gather_data(thread) for thread in tqdm(train_data)]

entire_dataset["val_prep"] = [gather_data(thread) for thread in tqdm(val_data)]

entire_dataset["test_prep"] = [gather_data(thread) for thread in tqdm(test_data)]

100%|████████████████████████████████████████████████████████████████████████████████| 258/258 [00:31<00:00,  8.27it/s]


In [11]:
def concatAllStringForBoW(listOfDict):
    return_obj = []
    for d_dict in listOfDict:
        return_obj = return_obj + list(d_dict.keys())
        
    return return_obj      

print(len(concatAllStringForBoW(entire_dataset["train_prep"])))

4837


In [12]:
train_bow_input = concatAllStringForBoW(entire_dataset["train_prep"])
val_bow_input = concatAllStringForBoW(entire_dataset["val_prep"])
test_bow_input = concatAllStringForBoW(entire_dataset["test_prep"])

print(len(train_bow_input))

4837


In [13]:
vectorizer = CountVectorizer()

train_bow = vectorizer.fit_transform(train_bow_input)
val_bow = vectorizer.fit(val_bow_input)
test_bow = vectorizer.fit(test_bow_input)

In [14]:
print(len(train_bow.toarray().tolist()))

4837


In [ ]:
def get_input_vector(key_string):
    tuple_counter = 0
    input_vector = []
    for i in range(len(entire_dataset[key_string])):
        feature_vals = list(entire_dataset[key_string][i].values())
        print(i, end=" ")
        thread_vector = []
        for j in range(len(feature_vals)):
            #print(tuple_counter + i + j)
            thread_vector.append( train_bow.toarray().tolist()[tuple_counter + i + j] + feature_vals[j] )
        #print(thread_vector)   
        tuple_counter = tuple_counter + j 
        input_vector.append(thread_vector)
        
    return input_vector


x_train = tqdm(get_input_vector('train_prep'))

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 

  0%|                                                                                       | 0/1936 [2:28:59<?, ?it/s]


27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 2

In [ ]:
x_val = get_input_vector('val_prep')
x_test = get_input_vector('test_prep')

In [ ]:
# random classification for baseline score

#random_val = {t_id: random.randint(0,1) for t_id in val_ids}
#random_test = {t_id: random.randint(0,1) for t_id in test_ids}


In [ ]:
#with open("val-random-output.json", "w") as f:
#    json.dump(random_val, f)
# get testing dataset 
#with open("test-random-output.json", "w") as f:
#    json.dump(random_test, f)
